In [6]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define paths
base_folder = r"C:\Users\Manas\IoT Research Models\ODIR-5K\ODIR-5K"
training_folder = os.path.join(base_folder, "Training Images")
testing_folder = os.path.join(base_folder, "Testing Images")
preprocessed_folder = os.path.join(base_folder, "Preprocessed_Images")  # New folder
csv_path = r"C:\Users\Manas\IoT Research Models\full_df.csv"  # Input Excel file
output_xlsx_path = r"C:\Users\Manas\IoT Research Models\full_df.xlsx"  # Output Excel file

# Create the folder if it doesn't exist
os.makedirs(preprocessed_folder, exist_ok=True)

# Function to preprocess images
def preprocess_image(image_path):
    try:
        filename = os.path.basename(image_path)  # Get image filename
        save_path = os.path.join(preprocessed_folder, filename)  # New destination

        img = cv2.imread(image_path)  # Read image
        if img is None:
            print(f"⚠️ Warning: Could not load {image_path}")
            return filename, None  # Skip if image is corrupted

        img = cv2.resize(img, (224, 224))  # Resize
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        img = img / 255.0  # Normalize

        # Convert to uint8 (optional for saving)
        img = (img * 255).astype(np.uint8)

        # Save processed image
        cv2.imwrite(save_path, img)

        return filename, save_path  # Return new path
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return filename, None

# Find all images in both Training and Testing folders
training_images = glob.glob(os.path.join(training_folder, "*.jpg")) + glob.glob(os.path.join(training_folder, "*.png"))
testing_images = glob.glob(os.path.join(testing_folder, "*.jpg")) + glob.glob(os.path.join(testing_folder, "*.png"))
all_images = training_images + testing_images  # Merge lists

# Process images in parallel with tqdm
new_image_paths = {}
with ThreadPoolExecutor(max_workers=8) as executor:  # Use 8 threads
    futures = {executor.submit(preprocess_image, img): img for img in all_images}
    
    for future in tqdm(as_completed(futures), total=len(futures), desc="Preprocessing Images", unit="img"):
        filename, result = future.result()
        if result:
            new_image_paths[filename] = result  # Store new paths

print(f"✅ All preprocessed images saved in: {preprocessed_folder}")

# Load Excel file and update file paths
original_df = pd.read_csv(csv_path)  # Read Excel file

# Update file paths in the dataset
tqdm.pandas(desc="Updating file paths")
original_df["Left-Fundus"] = original_df["Left-Fundus"].progress_apply(lambda x: new_image_paths.get(os.path.basename(x), None) if isinstance(x, str) else x)
original_df["Right-Fundus"] = original_df["Right-Fundus"].progress_apply(lambda x: new_image_paths.get(os.path.basename(x), None) if isinstance(x, str) else x)

# Drop any rows where images are missing after preprocessing
original_df.dropna(subset=["Left-Fundus", "Right-Fundus"], inplace=True)

# Save updated dataset as an Excel file
original_df.to_excel(output_xlsx_path, index=False)
print(f"✅ New Excel file saved at: {output_xlsx_path}")


Preprocessing Images: 100%|██████████████████████████████████████████████████████| 8000/8000 [00:40<00:00, 196.01img/s]


✅ All preprocessed images saved in: C:\Users\Manas\IoT Research Models\ODIR-5K\ODIR-5K\Preprocessed_Images


Updating file paths: 100%|█████████████████████████████████████████████████████| 6392/6392 [00:00<00:00, 220413.46it/s]


✅ New Excel file saved at: C:\Users\Manas\IoT Research Models\full_df.xlsx


In [32]:
original_df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,labels,target,filename
0,0,69,Female,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,cataract,normal fundus,0,0,0,1,0,0,0,0,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,normal fundus,normal fundus,1,0,0,0,0,0,0,0,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,C:\Users\Manas\IoT Research Models\ODIR-5K\ODI...,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [8]:
original_df = original_df.drop("filepath", axis = 1)

In [14]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    precision_recall_fscore_support
)
import time
import logging

# --------------------- Optional: Mixed Precision ---------------------
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
# ---------------------------------------------------------------------

# --------------------- Optimizations ---------------------
AUTOTUNE = tf.data.AUTOTUNE
SHUFFLE_BUFFER = 250  # Increased shuffle buffer for more randomness
# ---------------------------------------------------------------------

# Set up logging for any preprocessing errors
logging.basicConfig(filename="preprocessing_errors.log", level=logging.ERROR)

# Define paths
base_folder = r"C:\Users\Manas\IoT Research Models\ODIR-5K\ODIR-5K"
xlsx_path = r"C:\Users\Manas\IoT Research Models\full_df.xlsx"  # Dataset Excel file

# Load the dataset
df = pd.read_excel(xlsx_path)
labels_list = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
df['target'] = df[labels_list].idxmax(axis=1)

# Prepare image file paths and corresponding targets
left_paths = df['Left-Fundus'].dropna().tolist()
right_paths = df['Right-Fundus'].dropna().tolist()
all_paths = left_paths + right_paths
patient_targets = df['target'].dropna().tolist()
all_targets = patient_targets + patient_targets  # Duplicate for left & right images

# Filter out invalid paths
valid_paths = [p for p in all_paths if os.path.isfile(p)]
if len(valid_paths) == 0:
    raise ValueError("No valid images found. Please check the dataset.")

# Map textual labels to integer indices
label_to_index = {label: idx for idx, label in enumerate(labels_list)}
target_indices = [label_to_index[label] for label in all_targets]

# Split dataset into training and validation sets
X_train_paths, X_val_paths, y_train, y_val = train_test_split(
    valid_paths, target_indices, test_size=0.2, random_state=42
)
# One-hot encode labels
y_train = to_categorical(y_train, num_classes=8)
y_val = to_categorical(y_val, num_classes=8)

# --- Build tf.data Pipeline ---
IMG_SIZE = 224
BATCH_SIZE = 16  # Reduced batch size to reduce memory usage

def process_image(file_path, label):
    image = tf.io.read_file(file_path)
    try:
        image = tf.image.decode_jpeg(image, channels=3)
    except Exception as e:
        tf.print(f"Error decoding {file_path}: {e}")
        image = tf.zeros([IMG_SIZE, IMG_SIZE, 3], dtype=tf.uint8)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label

def augment(image, label):
    image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE+20, IMG_SIZE+20)
    image = tf.image.random_crop(image, size=[IMG_SIZE, IMG_SIZE, 3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

def create_dataset(file_paths, labels, training=True):
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    ds = ds.map(process_image, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
    ds = ds.map(normalize, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(SHUFFLE_BUFFER)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.cache().prefetch(AUTOTUNE)
    return ds

train_ds = create_dataset(X_train_paths, y_train, training=True)
val_ds = create_dataset(X_val_paths, y_val, training=False)

# Diagnostic: Check one batch
start = time.time()
for batch in train_ds.take(1):
    images_batch, labels_batch = batch
    print("One batch shape:", images_batch.shape)
end = time.time()
print("Time taken for one batch:", end - start, "seconds")

# --- Build the Model ---
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base model initially

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(8, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# --- Training ---
print("Starting training...")
start_time = time.time()
history = model.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)
training_time = time.time() - start_time

# Save the model
model.save("ocular_disease_resnet50_model.h5")
print("✅ Model saved as 'ocular_disease_resnet50_model.h5'")

# --- Evaluation ---
loss, val_accuracy = model.evaluate(val_ds, verbose=1)
print(f"✅ Validation Accuracy: {val_accuracy*100:.2f}%")

# Predictions for metrics
y_pred = model.predict(val_ds)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.concatenate([y for _, y in val_ds], axis=0), axis=1)

# Additional Metrics
roc_score = roc_auc_score(np.concatenate([y for _, y in val_ds], axis=0), y_pred, multi_class='ovr')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred_classes, average='weighted')

# Number of Parameters and Layers
num_params = model.count_params()
num_layers = len(model.layers)

print("\n--- Metrics for Comparative Study ---")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"ROC AUC Score: {roc_score:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Number of Parameters: {num_params}")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Number of Layers: {num_layers}")

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=labels_list))


One batch shape: (16, 224, 224, 3)
Time taken for one batch: 0.33202433586120605 seconds
Starting training...
Epoch 1/50
640/640 [==============================] - 65s 87ms/step - loss: 1.6654 - accuracy: 0.3650 - val_loss: 1.5605 - val_accuracy: 0.4138 - lr: 1.0000e-04
Epoch 2/50
640/640 [==============================] - 49s 77ms/step - loss: 1.5777 - accuracy: 0.3897 - val_loss: 1.5099 - val_accuracy: 0.4259 - lr: 1.0000e-04
Epoch 3/50
640/640 [==============================] - 55s 85ms/step - loss: 1.5449 - accuracy: 0.3970 - val_loss: 1.4947 - val_accuracy: 0.4333 - lr: 1.0000e-04
Epoch 4/50
640/640 [==============================] - 44s 69ms/step - loss: 1.5235 - accuracy: 0.3976 - val_loss: 1.4846 - val_accuracy: 0.4345 - lr: 1.0000e-04
Epoch 5/50
640/640 [==============================] - 43s 67ms/step - loss: 1.5040 - accuracy: 0.4128 - val_loss: 1.4687 - val_accuracy: 0.4364 - lr: 1.0000e-04
Epoch 6/50
640/640 [==============================] - 45s 71ms/step - loss: 1.4841 - 

In [17]:
metrics = {
    "Accuracy": val_accuracy,
    "ROC Score": roc_score,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1_score,
    "Number of Parameters": num_params,
    "Training Time (seconds)": training_time,
    "Number of Layers": num_layers
}

# Save metrics to a CSV file
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv("Model_metrics.csv", index=False)
print("✅ Metrics saved to 'Model_metrics.csv'")

✅ Metrics saved to 'Model_metrics.csv'


In [2]:
# Import necessary libraries
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model  # Correctly import load_model

# Load your model
model = load_model("ocular_disease_resnet50_model.h5")

# Save model summary to a text file
with open('model_summary_resnet50.txt', 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
print("Model summary saved to 'model_summary_vgg16.txt'.")

# Plot training curves
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Val Accuracy', marker='o')
plt.title("Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Val Loss', marker='o')
plt.title("Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.savefig('resnet50_eval.png')  # Save the training curves plot
print("Training curves saved to 'resnet50_eval.png'.")
plt.show()

Model summary saved to 'model_summary_vgg16.txt'.


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [1]:
import torch
print(torch.cuda.device_count())

1


In [1]:
import tensorflow as tf
import torch

# TensorFlow GPU check
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"✅ TensorFlow detected {len(gpus)} GPU(s).")
else:
    print("❌ TensorFlow did not detect any GPUs.")

# PyTorch GPU check
if torch.cuda.is_available():
    print(f"✅ PyTorch detected {torch.cuda.device_count()} GPU(s).")
    device = torch.device("cuda")
else:
    print("❌ PyTorch did not detect any GPUs.")
    device = torch.device("cpu")

C:\Users\Manas\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ TensorFlow detected 1 GPU(s).
❌ PyTorch did not detect any GPUs.


In [2]:
import tensorflow as tf
print("GPUs available:", tf.config.list_physical_devices('GPU'))


GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))


2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
import tensorflow as tf
print("Num GPUs Available (TensorFlow):", len(tf.config.list_physical_devices('GPU')))

import torch
print("PyTorch GPU Available:", torch.cuda.is_available())

Num GPUs Available (TensorFlow): 1
PyTorch GPU Available: False
